# Multi-Agents in LangChain 1.0
* To use sub-agents, just call them as tools from the main agent.

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain.agents import create_agent
from langchain.tools import tool

@tool
def square_root(x: float) -> float:
    """Calculate the square root of a number"""
    return x ** 0.5

@tool
def square(x: float) -> float:
    """Calculate the square of a number"""
    return x ** 2


from langchain.agents import create_agent

# create subagents

subagent_1 = create_agent(
    model='gpt-4o-mini',
    tools=[square_root]
)

subagent_2 = create_agent(
    model='gpt-4o-mini',
    tools=[square]
)

from langchain.messages import HumanMessage

@tool
def call_subagent_1(x: float) -> float:
    """Call subagent 1 in order to calculate the square root of a number"""
    response = subagent_1.invoke({"messages": [HumanMessage(content=f"Calculate the square root of {x}")]})
    return response["messages"][-1].content

@tool
def call_subagent_2(x: float) -> float:
    """Call subagent 2 in order to calculate the square of a number"""
    response = subagent_2.invoke({"messages": [HumanMessage(content=f"Calculate the square of {x}")]})
    return response["messages"][-1].content

## Creating the main agent

main_agent = create_agent(
    model='gpt-4o-mini',
    tools=[call_subagent_1, call_subagent_2],
    system_prompt="You are a helpful assistant who can call subagents to calculate the square root or square of a number.")


question = "What is the square root of 456?"

response = main_agent.invoke({"messages": [HumanMessage(content=question)]})

print(response['messages'][-1].content)

The square root of 456 is approximately 21.35.


## Using pprint we can see that the Agent is calling the subagent

In [3]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content='What is the square root of 456?', additional_kwargs={}, response_metadata={}, id='3fa6771a-c7f6-48f0-a3e2-13300d4ff0cc'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 118, 'total_tokens': 135, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_c4585b5b9c', 'id': 'chatcmpl-CuCfUWpW6R0d8n4eQn8b8lotLBgM3', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b87df-0947-7cb2-b87d-1331416a988e-0', tool_calls=[{'name': 'call_subagent_1', 'args': {'x': 456}, 'id': 'call_lGY2kT60FkCs9wjaKHeYocPg', 'type': 'tool_call'}], usage_metadata={'input_tokens': 118, 'output_tokens':

## OK. Let's explain the previous code in simple terms
Below is a **simple, beginner-friendly, line-by-line explanation** of what this LangChain 1.0 code does.
I’ll explain **what each part means and why it exists**, without assuming prior knowledge of LangChain or agents.

---

#### 1. Importing tools to build agents

```python
from langchain.agents import create_agent
from langchain.tools import tool
```

* `create_agent` is a helper that creates an AI **agent** (a model that can reason and use tools).
* `tool` is a decorator that turns a normal Python function into a **tool** the agent can call.

---

#### 2. Creating simple math tools

**Square root tool**

```python
@tool
def square_root(x: float) -> float:
    """Calculate the square root of a number"""
    return x ** 0.5
```

What’s happening here:

* `@tool` tells LangChain:

  > “This function can be used by an AI agent.”
* The function:

  * Takes a number (`x`)
  * Returns its square root using `x ** 0.5`
* The docstring helps the AI understand **what the tool does**

---

**Square tool**

```python
@tool
def square(x: float) -> float:
    """Calculate the square of a number"""
    return x ** 2
```

Same idea as before:

* This tool:

  * Takes a number
  * Returns its square (`x * x`)

---

#### 3. Creating sub-agents

```python
from langchain.agents import create_agent
```

This import is repeated (not harmful, just redundant).

---

**Sub-agent 1: square root agent**

```python
subagent_1 = create_agent(
    model='gpt-4o-mini',
    tools=[square_root]
)
```

This creates an AI agent that:

* Uses the model **gpt-4o-mini**
* Has access to **only one tool**: `square_root`

Think of it as:

> “An AI that only knows how to calculate square roots.”

---

**Sub-agent 2: square agent**

```python
subagent_2 = create_agent(
    model='gpt-4o-mini',
    tools=[square]
)
```

This agent:

* Uses the same model
* Can only calculate **squares**

So now you have **two specialized AI helpers**.

---

#### 4. Importing message format

```python
from langchain.messages import HumanMessage
```

* `HumanMessage` represents a message written by a user.
* Agents in LangChain communicate using message objects, not plain strings.

---

#### 5. Tools that call sub-agents

**Tool to call sub-agent 1 (square root)**

```python
@tool
def call_subagent_1(x: float) -> float:
    """Call subagent 1 in order to calculate the square root of a number"""
```

* This function itself is also a **tool**
* It doesn’t do math directly
* Instead, it **asks another agent to do it**

---

```python
response = subagent_1.invoke({
    "messages": [
        HumanMessage(content=f"Calculate the square root of {x}")
    ]
})
```

What this does:

* Sends a message to `subagent_1`
* The message looks like something a human would say
* The agent decides how to answer (usually by calling `square_root`)

---

```python
return response["messages"][-1].content
```

* The agent’s full conversation is returned
* `[-1]` means “the last message”
* `.content` extracts the actual text answer

---

**Tool to call sub-agent 2 (square)**

```python
@tool
def call_subagent_2(x: float) -> float:
    """Call subagent 2 in order to calculate the square of a number"""
```

This works exactly the same as `call_subagent_1`, but:

* It talks to `subagent_2`
* It asks for the **square**, not the square root

---

#### 6. Creating the main agent

```python
main_agent = create_agent(
    model='gpt-4o-mini',
    tools=[call_subagent_1, call_subagent_2],
    system_prompt="You are a helpful assistant who can call subagents to calculate the square root or square of a number."
)
```

This is the **boss agent**:

* It uses the same AI model
* It has access to:

  * `call_subagent_1`
  * `call_subagent_2`
* The `system_prompt` tells it:

  > “You don’t calculate directly. You delegate work to sub-agents.”

---

#### 7. Asking a question

```python
question = "What is the square root of 456?"
```

This is a normal user question.

---

```python
response = main_agent.invoke({
    "messages": [HumanMessage(content=question)]
})
```

What happens internally:

1. The main agent reads the question
2. It decides:

   * “This is a square root problem”
3. It calls `call_subagent_1`
4. That sub-agent calls the `square_root` tool
5. The result flows back up

---

#### 8. Printing the final answer

```python
print(response['messages'][-1].content)
```

* Extracts the final message from the agent
* Prints the answer to the console

---

#### 🧠 Big-picture summary

* You created **small math tools**
* You wrapped them into **specialized sub-agents**
* You created a **main agent** that:

  * Understands questions
  * Delegates work to the right sub-agent
* This mimics a **team of AIs**, each with a clear role

## How to run this code from Visual Studio Code
* Open Terminal.
* Make sure you are in the project folder.
* Make sure you have the poetry env activated.
* Enter and run the following command:
    * `python 009-multi-agents.py`